# NER using trained pipeline for Spanish

#### @mnsosa


In [184]:
from spacy import displacy
import spacy

## It's necessary to download the pipline. 

# _**python -m spacy download es_core_news_lg**_

In [185]:
nlp = spacy.load('es_core_news_lg')

In [186]:
# read() – read all text from a file into a string. 
# This method is useful if you have a small file and you want to manipulate the whole text of that file.
# readline() – read the text file line by line and return all the lines as strings.
# readlines() – read all the lines of the text file and return them as a list of strings. 
open('text_eg_1.txt', 'r').read()

'Este caso se refiere al asesinato de Jorge Omar Gutiérrez, de un disparo en la nuca, el 29 de\nagosto de 1994. Jorge era un subcomisario ejemplar de la Policía de la Provincia de Buenos\nAires, que fue asesinado por un agente de la Policía Federal Argentina y los responsables\nencubiertos por ambas fuerzas policiales. Al momento de los hechos, se encontraba\ninvestigando un depósito inmediatamente vecino a la comisaria donde se desempeñaba,\ncustodiado sospechosamente por miembros de la Policía Federal.\nA pesar de las incansables gestiones de sus familiares, después de casi dieciocho años, el\nEstado argentino no ha podido brindarles una respuesta judicial que les permita conocer la\nverdad de lo sucedido e identificar y sancionar a los responsables materiales e intelectuales\ndel crimen.\nEn este hecho -que terminó siendo sólo una parte de lo que se conoció como el "caso de la\nAduana Paralela"-, no sólo aparecen implicados en una compleja trama de ilegalidad,\nmiembros de la policí

In [187]:
with open('text_eg_1.txt') as f:
    text = f.read().strip('\n')

# Remove all \n
text_splitted = text.split('\n')
text_one_line = ""

for t in text_splitted:
    text_one_line = text_one_line + t + " "

doc = nlp(text_one_line)

In [188]:
doc.text

'Este caso se refiere al asesinato de Jorge Omar Gutiérrez, de un disparo en la nuca, el 29 de agosto de 1994. Jorge era un subcomisario ejemplar de la Policía de la Provincia de Buenos Aires, que fue asesinado por un agente de la Policía Federal Argentina y los responsables encubiertos por ambas fuerzas policiales. Al momento de los hechos, se encontraba investigando un depósito inmediatamente vecino a la comisaria donde se desempeñaba, custodiado sospechosamente por miembros de la Policía Federal. A pesar de las incansables gestiones de sus familiares, después de casi dieciocho años, el Estado argentino no ha podido brindarles una respuesta judicial que les permita conocer la verdad de lo sucedido e identificar y sancionar a los responsables materiales e intelectuales del crimen. En este hecho -que terminó siendo sólo una parte de lo que se conoció como el "caso de la Aduana Paralela"-, no sólo aparecen implicados en una compleja trama de ilegalidad, miembros de la policía federal si

In [189]:
displacy.render(doc, style='ent')

In [190]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Jorge Omar Gutiérrez 37 57 PER
Jorge 110 115 PER
Policía de la Provincia de Buenos Aires 151 190 ORG
Policía Federal Argentina 230 255 ORG
Al momento de los hechos 317 341 MISC
Policía Federal 487 502 ORG
Estado argentino 595 611 LOC
Aduana 882 888 ORG
Provincia de Buenos Aires 1040 1065 LOC
Carlos Gallone 1183 1197 PER
Jorge O. Gutiérrez 1242 1260 PER


# Anonymizing the text

## **IDEA**: 
### To copy and paste the paragraph between a start_char and a end_char for each PER entity. 
### Then replace the PER entity with "<NOMBRE>" (means 'name' in spanish).

In [191]:
doc.text[doc.ents[0].start_char:doc.ents[0].end_char]

'Jorge Omar Gutiérrez'

In [192]:
# The start_char of each entity PER
per_starts = [ent.start_char for ent in doc.ents if ent.label_ == 'PER']

# The end_char of each entity PER
per_ends = [ent.end_char for ent in doc.ents if ent.label_ == 'PER']

print(f'per_start: {per_starts}\nper_ends: {per_ends}')

per_start: [37, 110, 1183, 1242]
per_ends: [57, 115, 1197, 1260]


In [193]:
print(f"The start of the person 0: {per_starts[0]}")
print(f"The end of the person 0: {per_ends[0]}")

The start of the person 0: 37
The end of the person 0: 57


In [194]:
# Anonymization
text_anon = doc.text[0:per_starts[0]]
text_anon = text_anon + '<NOMBRE>'

per = 1
while per < len(per_starts):
    text_anon = text_anon + doc.text[per_ends[per-1]:per_starts[per]] + '<NOMBRE> '
    per = per + 1
    

In [195]:
text_anon

'Este caso se refiere al asesinato de <NOMBRE>, de un disparo en la nuca, el 29 de agosto de 1994. <NOMBRE>  era un subcomisario ejemplar de la Policía de la Provincia de Buenos Aires, que fue asesinado por un agente de la Policía Federal Argentina y los responsables encubiertos por ambas fuerzas policiales. Al momento de los hechos, se encontraba investigando un depósito inmediatamente vecino a la comisaria donde se desempeñaba, custodiado sospechosamente por miembros de la Policía Federal. A pesar de las incansables gestiones de sus familiares, después de casi dieciocho años, el Estado argentino no ha podido brindarles una respuesta judicial que les permita conocer la verdad de lo sucedido e identificar y sancionar a los responsables materiales e intelectuales del crimen. En este hecho -que terminó siendo sólo una parte de lo que se conoció como el "caso de la Aduana Paralela"-, no sólo aparecen implicados en una compleja trama de ilegalidad, miembros de la policía federal sino tambi

In [196]:
doc.text

'Este caso se refiere al asesinato de Jorge Omar Gutiérrez, de un disparo en la nuca, el 29 de agosto de 1994. Jorge era un subcomisario ejemplar de la Policía de la Provincia de Buenos Aires, que fue asesinado por un agente de la Policía Federal Argentina y los responsables encubiertos por ambas fuerzas policiales. Al momento de los hechos, se encontraba investigando un depósito inmediatamente vecino a la comisaria donde se desempeñaba, custodiado sospechosamente por miembros de la Policía Federal. A pesar de las incansables gestiones de sus familiares, después de casi dieciocho años, el Estado argentino no ha podido brindarles una respuesta judicial que les permita conocer la verdad de lo sucedido e identificar y sancionar a los responsables materiales e intelectuales del crimen. En este hecho -que terminó siendo sólo una parte de lo que se conoció como el "caso de la Aduana Paralela"-, no sólo aparecen implicados en una compleja trama de ilegalidad, miembros de la policía federal si

# Testing doc attributes
Just for learning :)

In [197]:
T = doc.text.split("\n")
T2 = ""

for t in T:
    T2 = T2 + t

T2

'Este caso se refiere al asesinato de Jorge Omar Gutiérrez, de un disparo en la nuca, el 29 de agosto de 1994. Jorge era un subcomisario ejemplar de la Policía de la Provincia de Buenos Aires, que fue asesinado por un agente de la Policía Federal Argentina y los responsables encubiertos por ambas fuerzas policiales. Al momento de los hechos, se encontraba investigando un depósito inmediatamente vecino a la comisaria donde se desempeñaba, custodiado sospechosamente por miembros de la Policía Federal. A pesar de las incansables gestiones de sus familiares, después de casi dieciocho años, el Estado argentino no ha podido brindarles una respuesta judicial que les permita conocer la verdad de lo sucedido e identificar y sancionar a los responsables materiales e intelectuales del crimen. En este hecho -que terminó siendo sólo una parte de lo que se conoció como el "caso de la Aduana Paralela"-, no sólo aparecen implicados en una compleja trama de ilegalidad, miembros de la policía federal si

In [198]:
doc.tensor.shape

(317, 96)

In [199]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7fad4ad09f90>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7fad4ad25770>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7fad4acf5580>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7fad4acd5380>),
 ('lemmatizer',
  <spacy.lang.es.lemmatizer.SpanishLemmatizer at 0x7fad4ac74100>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fad4ad7df40>)]

In [200]:
len(doc.text.split(" "))

297

In [201]:
doc.text

'Este caso se refiere al asesinato de Jorge Omar Gutiérrez, de un disparo en la nuca, el 29 de agosto de 1994. Jorge era un subcomisario ejemplar de la Policía de la Provincia de Buenos Aires, que fue asesinado por un agente de la Policía Federal Argentina y los responsables encubiertos por ambas fuerzas policiales. Al momento de los hechos, se encontraba investigando un depósito inmediatamente vecino a la comisaria donde se desempeñaba, custodiado sospechosamente por miembros de la Policía Federal. A pesar de las incansables gestiones de sus familiares, después de casi dieciocho años, el Estado argentino no ha podido brindarles una respuesta judicial que les permita conocer la verdad de lo sucedido e identificar y sancionar a los responsables materiales e intelectuales del crimen. En este hecho -que terminó siendo sólo una parte de lo que se conoció como el "caso de la Aduana Paralela"-, no sólo aparecen implicados en una compleja trama de ilegalidad, miembros de la policía federal si

In [202]:
doc.ents

(Jorge Omar Gutiérrez,
 Jorge,
 Policía de la Provincia de Buenos Aires,
 Policía Federal Argentina,
 Al momento de los hechos,
 Policía Federal,
 Estado argentino,
 Aduana,
 Provincia de Buenos Aires,
 Carlos Gallone,
 Jorge O. Gutiérrez)

In [203]:
doc.vector.shape

(300,)

# Bibliography
<https://spacy.io/models/es#es_dep_news_trf>

<https://www.youtube.com/watch?v=uoBD3XjGD6s>

<https://towardsdatascience.com/ner-with-transformers-and-spacy-b3240bc65eb4>